<div class="alert alert-block alert-info">
<span style="color: rgb(0,53,91);">
<center><img src="https://www.redage.org/sites/default/files/styles/large/public/img-logo-institucion/logo_iteso.jpg?itok=IzloBJS1" style="width:500px;height:142px;" title="Logo ITESO"></center>

<font face = "Times New Roman" size = "6"><b><center>Programacion para Mineria de Datos</center></b></font>
<font face = "Times New Roman" size = "5"><b><center>Preprocessing Production</center></b></font>

<b><font face = "Times New Roman" size = "5"><center>Final Proyect</center></font>
<div align="right"><font face = "Times New Roman" size = "2">nombres: Angel Aceves Manzo</font></div>
<div align="right"><font face = "Times New Roman" size = "2"> Jorge Arturo Ponce Martin</font></div>
<div align="right"><font face = "Times New Roman" size = "2"> Jesus Vargas Pacheco</font></div>
</span></div>

In [1]:
#####################################################
#
# APLICAR PCA A DATOS QUE NUNCA HA VISTO
#
#####################################################
# Deben cargarse los archivos
# - preprocessor_cat.joblib
# - pca_pipe_num.joblib
# - pca_metadata.json
# - T_train_final_objetivo.csv
# - csv de tus nuevos datos
# Devolverá T_new_final.csv: el csv de PCA aplicado a los nuevos datos

# === Cargar artefactos para inferencia ===
import joblib
import json
import pandas as pd
#####################################################
preprocessor_cat = joblib.load("../preprocessing/preprocessor_cat.joblib")
pca_pipe = joblib.load("../preprocessing/pca_pipe_num.joblib")
with open("../preprocessing/pca_metadata.json", "r") as f:
#####################################################
    meta = json.load(f)

cols_num = meta["cols_num"]
cols_cat = meta["cols_cat"]
pc_cols  = meta["pc_cols"]
cat_out_cols = meta["cat_out_cols"]

# PCA DE ENTRENAMIENTO CON OBJETIVO
#####################################################
entrenamiento = pd.read_csv("../preprocessing/T_train_final_objetivo.csv")   ### todas las pca, categoricas y objetivo
print(entrenamiento)
#####################################################
entrenamiento_pca_objetivo = entrenamiento[pc_cols]



           PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0    -0.276920 -0.642359 -0.393789 -0.097924 -0.151540  0.721599 -0.058089   
1     1.645793 -0.800673  0.459823  0.397383  0.556896  0.110364  0.320271   
2     1.202848  0.213202  0.426680 -0.021795 -0.890292  0.022758 -0.459030   
3    -1.907295 -0.095893 -0.165868  0.975658 -0.456905 -1.204620 -0.732465   
4    -0.279265 -0.559393 -0.302561  0.243915 -0.678462  0.553153  0.105606   
...        ...       ...       ...       ...       ...       ...       ...   
1283  1.812703  1.038969  0.642752 -0.606935 -1.453148 -0.243004 -1.500107   
1284  4.385456 -1.172668  1.899900  1.267887  3.673244  1.371523 -0.845035   
1285 -0.780921 -0.779591 -1.144407  0.016822 -0.393156  2.758696 -0.824966   
1286  0.076002 -1.437209 -0.229529  0.268876  0.183010  0.556114  0.610828   
1287  5.882031 -0.257468  0.215232  0.322656  3.373781 -0.597518  0.214272   

           PC8  heating___electric  heating___hot air  ...  sew

In [2]:

# NUEVOS DATOS
#####################################################
new_df = pd.read_csv("../../data/Saratoga_desconocidos.csv")

#####################################################
X_new_num = new_df[cols_num]
X_new_cat = new_df[cols_cat]

# Categóricas (mismo encoder, sin re-ajustar)
X_new_cat_proc = preprocessor_cat.transform(X_new_cat)
df_new_cat_encode = pd.DataFrame(X_new_cat_proc, columns=cat_out_cols, index=new_df.index)

# Numéricas → (preproc_num + StdScaler + PCA) con el pipeline guardado
T_new = pca_pipe.transform(X_new_num)
T_new_df = pd.DataFrame(T_new, columns=pc_cols, index=new_df.index)

# Final
T_new_final = pd.concat([T_new_df, df_new_cat_encode], axis=1)
T_new_final.to_csv("T_new_final.csv",index=False)

In [3]:
import plotly.express as px
import plotly.graph_objects as go

# --- 1) figura base: train coloreado por objetivo (igual que antes) ---
y_train = entrenamiento["objetivo"]
df_plot2 = entrenamiento_pca_objetivo.iloc[:, :2].copy()
df_plot2["objetivo"] = y_train.loc[entrenamiento_pca_objetivo.index].astype(str)

fig = px.scatter(
    df_plot2, x="PC1", y="PC2", color="objetivo",
    opacity=0.85, title="PC1 vs PC2 — Train (objetivo) + Nuevos",
    height=550
)
fig.update_traces(marker=dict(size=6))

# --- 2) overlay: nuevos (un solo trazo, símbolo distinto) ---
new2 = T_new_df.loc[:, ["PC1", "PC2"]].copy()
fig.add_trace(
    go.Scatter(
        x=new2["PC1"], y=new2["PC2"],
        mode="markers",
        name="nuevos",
        marker=dict(size=9, line=dict(width=1)),
        opacity=0.95,
        showlegend=True
    )
)

fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")
fig.show()


In [4]:
T_new_final

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,heating___electric,heating___hot air,...,fuel___oil,sewer___none,sewer___public/commercial,sewer___septic,waterfront___No,waterfront___Yes,newConstruction___No,newConstruction___Yes,centralAir___No,centralAir___Yes
0,-1.905764,0.266144,1.022320,0.168308,-0.258120,-0.460221,0.535150,0.658734,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
1,1.159719,-0.761804,-0.909746,0.376705,-0.876126,-0.042257,0.401184,-0.496953,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,2.898206,-0.079321,-0.177984,-0.608690,-0.464261,-0.411519,-0.478531,-0.768703,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1.737947,0.421540,0.292222,-0.020164,-0.664596,0.260617,0.535103,0.279524,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1.872174,-0.081409,0.664935,0.486500,-0.750387,1.817104,-0.407539,0.576972,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,2.604294,0.842840,0.212242,-0.822322,-1.252775,-0.201655,-0.321889,0.262750,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,2.477326,0.845647,-0.011809,0.120092,-0.586127,-0.100111,0.089898,-0.575221,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7,-2.868784,-0.813204,-0.410343,0.253822,0.446382,-0.362790,-0.243518,0.251302,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
8,-1.198604,0.472026,0.376708,-0.123120,-0.461421,1.070770,-0.945298,-0.247469,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9,-1.436986,1.870052,0.903307,-0.068640,0.261806,0.006589,0.228083,-0.149723,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [5]:
new_df

,lotSize,age,landValue,livingArea,pctCollege,bedrooms,fireplaces,bathrooms,rooms,heating,fuel,sewer,waterfront,newConstruction,centralAir,price
0,0.28,57,23200,1121,61,3,0,1.5,4,hot air,oil,septic,No,No,No,NaN
1,0.81,0,7300,2288,61,3,1,2.5,8,hot air,gas,public/commercial,No,Yes,No,NaN
2,0.41,5,43400,2718,61,4,1,2.5,12,hot air,gas,public/commercial,No,No,Yes,NaN
3,0.58,42,31400,2464,61,4,1,2.5,8,hot water/steam,gas,public/commercial,No,No,No,NaN
4,0.43,41,45800,2426,61,4,2,2.0,7,hot water/steam,gas,public/commercial,No,No,No,NaN
5,0.38,32,24200,2310,61,5,1,2.5,11,hot water/steam,gas,public/commercial,No,No,No,NaN
6,0.94,37,36200,2564,61,4,1,2.5,11,hot water/steam,gas,public/commercial,No,No,No,NaN
7,0.39,9,20400,1099,51,2,0,1.0,3,hot air,gas,public/commercial,No,No,No,NaN
8,0.24,48,16800,1225,51,3,1,1.0,7,hot air,gas,public/commercial,No,No,No,NaN
9,0.59,86,26000,1959,51,3,0,1.0,6,hot air,gas,septic,No,No,No,NaN


In [6]:
df_merged = pd.merge(new_df,T_new_final,left_index=True,right_index=True)
df_merged

,lotSize,age,landValue,livingArea,pctCollege,bedrooms,fireplaces,bathrooms,rooms,heating,...,fuel___oil,sewer___none,sewer___public/commercial,sewer___septic,waterfront___No,waterfront___Yes,newConstruction___No,newConstruction___Yes,centralAir___No,centralAir___Yes
0,0.28,57,23200,1121,61,3,0,1.5,4,hot air,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.81,0,7300,2288,61,3,1,2.5,8,hot air,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.41,5,43400,2718,61,4,1,2.5,12,hot air,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,0.58,42,31400,2464,61,4,1,2.5,8,hot water/steam,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.43,41,45800,2426,61,4,2,2.0,7,hot water/steam,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,0.38,32,24200,2310,61,5,1,2.5,11,hot water/steam,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,0.94,37,36200,2564,61,4,1,2.5,11,hot water/steam,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7,0.39,9,20400,1099,51,2,0,1.0,3,hot air,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
8,0.24,48,16800,1225,51,3,1,1.0,7,hot air,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9,0.59,86,26000,1959,51,3,0,1.0,6,hot air,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [7]:
import plotly.express as px
import plotly.graph_objects as go

# base: train 3D
df3 = entrenamiento_pca_objetivo.iloc[:, :3].copy()
df3["objetivo"] = y_train.loc[entrenamiento_pca_objetivo.index].astype(str)

fig3 = px.scatter_3d(
    df3, x="PC1", y="PC2", z="PC3",
    color="objetivo", opacity=0.85,
    title="PC1–PC2–PC3 — Train (objetivo) + Nuevos",
    height=600
)
fig3.update_traces(marker=dict(size=5))

# overlay: nuevos 3D
new3 = T_new_df.loc[:, ["PC1","PC2","PC3"]].copy()
fig3.add_trace(
    go.Scatter3d(
        x=new3["PC1"], y=new3["PC2"], z=new3["PC3"],
        mode="markers",
        name="nuevos",
        marker=dict(size=6, line=dict(width=1)),
        opacity=0.95,
        showlegend=True
    )
)

fig3.update_layout(scene=dict(xaxis_title="PC1", yaxis_title="PC2", zaxis_title="PC3"))
fig3.show()
